# ADVANCED TEXT ANALYTICS 2024/2025

## Scope of the project
Starting from a pre-trained model, the goal of the project is to attach a trained [ner](https://spacy.io/api/entityrecognizer) component to the model such that it will recognize labels coming from the medical field. The code is based on the spaCy Python library ([documentation here](https://spacy.io/api/doc)).

To address the ["catastrophic forgetting" problem](https://en.wikipedia.org/wiki/Catastrophic_interference), the trained ner component will be attached to a pre-trained model, the same one used for training the component, so that the output of the model will contain labels that can be assigned either by the original ner or by the trained ner component. Another possible solution could be performing a ["rehease"](https://spacy.io/api/language#rehearse), but in this project it is not explored.

<a id='step0'></a>

### STEP 0: install required libraries and check  for the GPU
Remove the comments to install the libraries required for running this notebook. 

In [ ]:
# %pip install spacy

In [ ]:
# %pip install tensorflow

In [ ]:
# %pip install pandas

In [ ]:
# %pip install spacy-transformers

In [ ]:
# !python -m spacy download en_core_web_trf

# Probably it is required to restart the Jupyter kernel after this instruction

### Installations for NVIDIA GPU

In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
# %pip install cupy-wheel

In [1]:
import tensorflow as tf

# Check version of tensorflow and if GPU is available
print(tf.__version__, tf.config.list_physical_devices('GPU'))

2.18.0 []


### Download required models

The models used in this project are:
- `en_core_web_lg`: English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer. Docs: [en_core_web_lg](https://spacy.io/models/en#en_core_web_lg)
- `en_core_web_trf`: English transformer pipeline (Transformer(name=‘roberta-base’, piece_encoder=‘byte-bpe’, stride=104, type=‘roberta’, width=768, window=144, vocab_size=50265)). Components: transformer, tagger, parser, ner, attribute_ruler, lemmatizer. Docs: [en_core_web_trf](https://spacy.io/models/en#en_core_web_trf)

In [ ]:
# !python -m spacy download en_core_web_lg

In [ ]:
# !python -m spacy download en_core_web_trf

<a id='step1'></a>

### STEP 1: prepare training set and dev set

In this project we define the **Annotations** that contains the annotations of the articles, and the **Articles** that contains the articles.

To upload the data use the following logic:
- Store the Annotations Train data inside `./Annotations/Train` folder;
- Store the Annotations Dev data inside `./Annotations/Dev` folder.
- Store the Articles Train data inside `./Articles/Train` folder;
- Store the Articles Dev data inside `./Articles/Dev` folder.

[DocBin](https://spacy.io/api/docbin) is used to store and serialize the Doc objects. The train DocBin will be saved in the `./TrainDocBin/train.spacy` file and the dev DocBin in the `./DevDocBin/dev.spacy` file.

**Note:** go to [step 2](#step2.0) if you already have the train and dev set well formatted.

In [1]:
from pathlib import Path
import os

path = str(Path(os.path.abspath(os.getcwd())).absolute())

# print(path)

# Open training set
with open(path + '/Articles/Train/articles_train.txt','r', encoding='UTF-8') as articlesTrainFile:
  articlesTrain = articlesTrainFile.read().split('\n\n')
  # Remove last empty line if present
  if articlesTrain[len(articlesTrain)-1] == '\n':
    articlesTrain = articlesTrain[:len(articlesTrain)-1]

with open(path + '/Annotations/Train/entities_train.txt','r', encoding='UTF-8') as entitiesTrainFile:
  entitiesTrainFile.readline()
  entitiesTrain = entitiesTrainFile.read().split('\n\n')
  # Remove last empty line if present
  if entitiesTrain[len(entitiesTrain)-1] == '\n':
    entitiesTrain = entitiesTrain[:len(entitiesTrain)-1]

with open(path + '/Articles/Dev/articles_dev.txt','r', encoding='UTF-8') as articlesDevFile:
  articlesDev = articlesDevFile.read().split('\n\n')
  if articlesDev[len(articlesDev)-1] == '\n':
    articlesDev = articlesDev[:len(articlesDev)-1]

with open(path + '/Annotations/Dev/entities_dev.txt','r', encoding='UTF-8') as entitiesDevFile:
  entitiesDevFile.readline()
  entitiesDev = entitiesDevFile.read().split('\n\n')
  if entitiesDev[len(entitiesDev)-1] == '\n':
    entitiesDev = entitiesDev[:len(entitiesDev)-1]

In [2]:
import re

def get_article(text):
  article = re.findall(r'a\|(.*)',text)
  return article[0]

def get_title(text):
  # print(text)
  title = re.findall(r't\|(.*)',text)
  return title[0]

def get_pmid(text):
  pmid = text.split('|', 1)[0]
  pmid = re.sub('\n', '', pmid)
  return pmid

def calc_article_start(text):
  title = re.findall(r't\|(.*)',text)
  return len(title[0]) + 1 # +1 because of space char added between title and abstract

# Articles for train data
train_id = [get_pmid(x) for x in articlesTrain]
train_articles = [get_title(x)+' '+get_article(x) for x in articlesTrain]
train_articles_start_at = [calc_article_start(x) for x in articlesTrain]

# Articles for test data
dev_id = [get_pmid(x) for x in articlesDev]
dev_articles = [get_title(x)+' '+get_article(x) for x in articlesDev]
dev_articles_start_at = [calc_article_start(x) for x in articlesDev]

In [3]:
import pandas as pd

train_df = pd.DataFrame(columns=['article', 'articleStartsAt'])
train_df['pmid'] = train_id
train_df['article'] = train_articles
train_df['articleStartsAt'] = train_articles_start_at

dev_df = pd.DataFrame(columns=['article'])
dev_df['pmid'] = dev_id
dev_df['article'] = dev_articles
dev_df['articleStartsAt'] = dev_articles_start_at

In [ ]:
train_df.head()

In [ ]:
dev_df.head()

In [4]:
def get_labels(dataframe, text):
  text = text.strip() # Remove possible \n at the start/end of the text
  l = text.split("\n")
  l = [x.split('\t') for x in l]
  labels = []
  index = 0
  for i in l:
    # print(i)
    while dataframe.iloc[index]['pmid'] != i[0]:
      index += 1
      continue

    if i[4] == 'title':
      labels.append((int(i[2]), int(i[3])+1, i[6]))
    elif i[4] == 'abstract':
      # Add shift due to the title length
      labels.append((int(i[2]) + int(dataframe.iloc[index]['articleStartsAt']), int(i[3]) + 1 + int(dataframe.iloc[index]['articleStartsAt']), i[6]))
  return labels

train_labels = [get_labels(train_df, x) for x in entitiesTrain]
dev_labels = [get_labels(dev_df, x) for x in entitiesDev]

print('total train labels: ', len(train_labels), ', total test labels: ' , len(dev_labels))

total train labels:  1566 , total test labels:  40


In [5]:
train_df['labels'] = train_labels
dev_df['labels'] = dev_labels

In [ ]:
train_df.head()

In [ ]:
dev_df.head()

In [6]:
training_data = []
for i, j in zip(train_articles, train_labels):
  training_data.append((i, j))

# print(training_data[0])

dev_data = []
for i, j in zip(dev_articles, dev_labels):
  dev_data.append((i, j))

# print(dev_data[0])

In [14]:
import spacy
import spacy.training

nlp = spacy.load("en_core_web_lg")

<a id='step1.1'></a>

### STEP 1.1: improve Tokenizer

Sometimes a word followed by a punctuation (or viceversa) are not considered as two distinct tokens. To fix this, a custom tokenizer is implemented by adding the list of puctuations to the default prefixes and suffixes lists.

In [ ]:
# Customizing the tokenizer
from spacy.tokenizer import Tokenizer
import re
import string

def custom_tokenizer(nlp):
  my_punct = f"[{re.escape(string.punctuation)}]"
  # my_punct = my_punct.replace("<", "")
  # my_punct = my_punct.replace(">", "")
  # html_tag_pattern = re.compile(r'<[^>]+>')

  all_prefixes_re = spacy.util.compile_prefix_regex(tuple(list(nlp.Defaults.prefixes) + [html_tag_pattern.pattern] + [my_punct]))
  infix_re = spacy.util.compile_infix_regex(nlp.Defaults.infixes)
  suffix_re = spacy.util.compile_suffix_regex(tuple(list(nlp.Defaults.suffixes) + [html_tag_pattern.pattern] + [my_punct]))

  
  return Tokenizer(nlp.vocab,
                   nlp.Defaults.tokenizer_exceptions,
                   prefix_search = all_prefixes_re.search, 
                   infix_finditer = infix_re.finditer,
                   suffix_search = suffix_re.search,
                   token_match=None)

nlp.tokenizer = custom_tokenizer(nlp=nlp)

In [ ]:
# Updated version of the tokenizer
from spacy.tokenizer import Tokenizer
import re
import string

def custom_tokenizer(nlp):
  prefix_re = re.compile(r'[.,;:?!(\[\'"</]')
  suffix_re = re.compile(r'[.,;:?!)\]\'">■™]')
  infix_re = re.compile(r'[-/+=&]')

  return Tokenizer(
        nlp.vocab,
        prefix_search=prefix_re.search,
        suffix_search=suffix_re.search,
        infix_finditer=infix_re.finditer
    )

nlp.tokenizer = custom_tokenizer(nlp=nlp)

In [16]:
from spacy.tokens import DocBin

def save_to_disk(data, dir, filename):
  db = DocBin()
  for text, annotations in data:
    doc = nlp(text)
    # sentence_tokens = []
    # for sent in doc.sents:
    #   sentence_tokens.append([token.text for token in sent])
    # print(sentence_tokens)
    ents = []
    for start, end, label in annotations:
      span = doc.char_span(int(start), int(end), label=label)
      if not span == None:
        ents.append(span)
      else:
        print("none", int(start), int(end), label)
    # print(ents)
    doc.ents = ents
    db.add(doc)

  db.to_disk(os.path.join(dir, filename))

In [17]:
save_to_disk(training_data, os.path.join(path,'TrainDocBin'), "train_last_lg.spacy")
save_to_disk(dev_data, os.path.join(path,'DevDocBin'), "dev_last_lg.spacy")

none 1341 1351 bacteria
none 1140 1145 statistical technique
none 806 820 DDF
none 1480 1499 DDF
none 1614 1632 DDF
none 1312 1354 bacteria
none 141 158 DDF
none 159 161 DDF
none 1635 1648 bacteria
none 1711 1725 bacteria
none 1529 1547 bacteria
none 504 508 chemical
none 1278 1293 biomedical technique
none 1312 1315 statistical technique
none 1829 1831 DDF
none 2147 2170 bacteria
none 128 130 DDF
none 941 945 chemical
none 1060 1065 chemical
none 717 721 DDF
none 1034 1050 bacteria
none 1067 1079 bacteria
none 1096 1117 bacteria
none 1134 1149 bacteria
none 1166 1182 bacteria
none 92 95 gene
none 579 596 anatomical location
none 439 445 chemical
none 1091 1094 biomedical technique
none 1110 1121 bacteria
none 1067 1070 drug
none 1220 1222 DDF
none 1223 1225 DDF
none 1777 1779 DDF
none 1780 1782 DDF
none 2112 2114 DDF
none 2115 2117 DDF
none 2422 2424 DDF
none 2425 2427 DDF
none 167 170 DDF
none 359 381 human
none 788 791 DDF
none 806 828 human
none 332 334 DDF
none 1588 1607 DDF
none 

<a id='step2'></a>

### STEP 2: prepare CUDA and PyTorch

If your PC is already set up correctly, then skip to [step 3](#step3).

#### Check if CUDA is available
The instruction *torch.cuda.is_available()* checks if CUDA is avaiable for running the train on the GPU.
If the answer if false, then it means either PyTorch or CUDA or both of them is not installed.

#### Install PyTorch
To install PyTorch, go to [this link](https://pytorch.org/get-started/locally/), select your preferences (in this case it is important to set a CUDA version as "Compute Platform" so that the code will run on the GPU) and then copy-paste the command into the following cell.

It might be necessary to restart the runtime.

After installing pythorch, *torch.cuda.is_available()* returns true.

In [ ]:
# Check if CUDA is available
import torch

print('is cuda available? ', torch.cuda.is_available())
print(torch.__version__)
!nvidia-smi

<a id='step3'></a>

### STEP 3: train the NER component

#### Generate config.cfg file
Generate the base_config.cfg configuration file that includes all the settings and hyperparameters.
In this project the focus is to train only the ner component.
The train will be optimized for accuracy over efficiency.
Then, save the config to config.cfg file

For this project the training is done with an NVIDIA GeForce 4060 laptop with 8GB of VRAM. 

#### Train en_core_web_trf NER component

In [50]:
!python -m spacy init fill-config ./config/trf/base_config.cfg ./config/trf/config.cfg

✔ Auto-filled config with all values
✔ Saved config
config\trf\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# !python -m prodigy spacy-config config.cfg --ner custom_tok_annotations --base-model custom_tokenizer

c:\Users\Manuel\ata-project\Scripts\python.exe: No module named prodigy


In [ ]:
# import spacy
# import spacy.training

# # Load pre-trained model over which a new training for the ner component will be done
# nlp = spacy.load("en_core_web_trf")

In [ ]:
!python -m spacy train ./config/trf/config.cfg --output ./output/trf-improve-tokenizer --paths.train ./TrainDocBin/train.spacy --paths.dev ./DevDocBin/dev.spacy --gpu-id 0

ℹ Saving to output directory: output\trf-improve-tokenizer
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0         909.92   1280.13    0.21    0.12    0.81    0.00
  0     200      196977.51  78137.02   67.24   65.08   69.56    0.67
  1     400       48385.19  24469.14   79.55   77.55   81.65    0.80
  2     600       16277.44  18497.41   77.29   74.65   80.13    0.77
  3     800        9208.31  15503.75   80.71   77.76   83.89    0.81
  3    1000        8040.32  14244.09   80.91   79.18   82.72    0.81
  4    1200        6980.84  12304.42   78.61   75.41   82.09    0.79
  5    1400        6619.55  11215.05   80.15   81.37   78.96   

c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Manuel\ata-project\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_p

In [ ]:
unique_labels = set()

for example in dev_data:
	entities = example[1]
	for entity in entities:
		entity_label = entity[2]
		unique_labels.add(entity_label)
        
unique_labels_list = list(unique_labels)

print('Entities to be recognised in the provided data: ')
print(unique_labels_list)

### STEP 3.1: evaluate best model

In [1]:
!spacy evaluate ./output/trf-improve-tokenizer/model-best/ ./DevDocBin/dev.spacy --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK     99.74
NER P   78.89
NER R   83.97
NER F   81.35
SPEED   1331 


=============================== NER (per type) ===============================

                            P       R       F
microbiome              86.52   96.06   91.04
DDF                     86.20   87.34   86.76
drug                    89.47   85.00   87.18
bacteria                70.91   72.22   71.56
human                   92.31   97.67   94.92
anatomical location     83.33   92.11   87.50
chemical                58.38   82.44   68.35
biomedical technique    53.49   63.89   58.23
gene                    30.43   17.95   22.58
statistical technique   33.33   66.67   44.44
animal                  94.52   94.52   94.52
dietary supplement      61.76   77.78   68.85
food                    84.62   42.31   56.41



c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Manuel\ata-project\Lib\site-packages\spacy_transformers\layers\hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implici

<a id='step3'></a>

### STEP 3: attach the trained NER comopnent to the original model
In the following cell, the trained NER component will be integrated into the original en_core_web_trf model. This combination will allow the final model to label words using a set that includes both the original labels and the newly trained ones.

In [ ]:
# Load original model
nlp = spacy.load("en_core_web_trf")
# Load trained model
trained_nlp = spacy.load('./output/model-best')

trained_nlp.replace_listeners("transformer", "ner", ["model.tok2vec"])

nlp.add_pipe(
  "ner",
  name="trained_ner",
  source=trained_nlp,
  before="ner",
)

print(nlp.pipe_names)

In [ ]:
import spacy.displacy


doc = nlp("Doc here")
spacy.displacy.render(doc, style="ent", jupyter=True)

STEP 4: train using the en_core_web_lg model

### FINAL: output of best models

| Pipeline | .spacy creation | Params | TOK | NER P | NER R | NER F |
| --- | --- | --- | --- | --- | --- | --- |
| en_core_web_trf | Base | Base | 99.90 | 83.58 | 80.20 | 81.86 |
| en_core_web_lg | Base | Base | 99.90 | 81.62 | 72.73 | 76.92 |
| en_core_web_lg | improved tokenizer | Base | 99.74 | 78.29 | 74.57 | 76.39 |
| en_core_web_trf | updated tokenizer | Base + change config | 90.71 | 80.98 | 81.56 | 81.27 |
| en_core_web_trf | improved tokenizer | Base + change config | 99.74 | 78.89 | 83.97 | 81.35 |